In [8]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupShuffleSplit


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts

# Set display options to show all rows and columns
pd.set_option('display.max_rows', 50)  # Show rows
pd.set_option('display.max_columns', 160)  # Show columns

In [36]:
# 600 times slower!!!! :O 
4*3*4*2*5

480

In [42]:
# Quicker version of param grid
param_grid = { 
    'n_estimators': [10, 40, 100, 250],
    'max_features': [None, 'sqrt'],
    'max_depth' : [3,5,8, None],
    'criterion' :['gini'] }
#CV to choose hyperparameters automtatically adds a x5 at the end 
4*2*4*1*5

160

In [39]:
#Example Parameter grid for Random Forest
param_grid = { 
    'n_estimators': [10, 30, 100, 200, 500],
    'max_features': [None, 'sqrt', 'log2'],
    'max_depth' : [3,5,8, None],
    'criterion' :['gini', 'entropy'] }

clf =  GridSearchCV( RandomForestClassifier(), param_grid, refit = True, verbose = 1)

 ### Load the static features (Wake data testing)

In [9]:
core_path = '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Baseline_Extensions/Gen_New_Features/generated_feats/'
data_types = ['N2', 'N3','REM', 'Wake', 'N1']
data_type = 'Wake' # Select data_type to load

load_path = core_path + data_type

X = pd.read_hdf(load_path + 'six_second_max_freq_stats_df.h5', key='df', mode='r')
y = pd.read_hdf(load_path + '_y.h5', key='df', mode='r') 
groups = pd.read_hdf(load_path + '_groups.h5', key='df', mode='r')  

# Replace time sereis data with mean of the data ______________________________________________________________________________________________________

# Defining a function to replace time series of values with their mean
def function(x):
    return x.values.mean()
    
#Construct the static features
static_features_df = X.apply(np.vectorize(function))
X = static_features_df.copy()

In [30]:
Prefrontal_only_dict = {'Prefrontal' : [col for col in X.columns if 'Prefrontal' in col] }

In [50]:
Prefrontal_Left_Temporal_dict = {'Prefrontal_Left_Temporal' : [col for col in X.columns if 'Prefrontal' in col] + [col for col in X.columns if 'Left Temporal' in col] }

In [18]:
#Generate region to features dictionary to enable experiments to be run regionally
regional_features_dict = {}
region_channel_dict = constants.region_to_channel_dict
regions = list(region_channel_dict.keys())
for region in regions:
    region_features = [col for col in X.columns if '_' + region in col]
    if len(region_features) > 0 : 
        regional_features_dict[region] = region_features

#### Run experiment test

In [51]:
t1 = time.time()

X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 1 )
results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : GridSearchCV( RandomForestClassifier(), param_grid, refit = True, verbose = 1)}, return_df = True , subset_names_and_cols = Prefrontal_Left_Temporal_dict, random_states = [1,2] )
# results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier()}, return_df = True , subset_names_and_cols = Prefrontal_only_dict, random_states = [1,2] ) 

res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
display(pd.DataFrame(res_df))
        
t2 = time.time()

t2-t1

Fitting 5 folds for each of 32 candidates, totalling 160 fits
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Fitting 5 folds for each of 32 candidates, totalling 160 fits


,random_state,Prefrontal_Left_Temporal_RF_acc,Prefrontal_Left_Temporal_RF_std
0,1,0.733333,0.169967
1,2,0.633333,0.244949


136.15371227264404

In [52]:
t1 = time.time()

X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 2)
results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : GridSearchCV( RandomForestClassifier(), param_grid, refit = True, verbose = 1)}, return_df = True , subset_names_and_cols = Prefrontal_Left_Temporal_dict, random_states = [1,2] ) 
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
display(pd.DataFrame(res_df))
        
t2 = time.time()

t2-t1

Fitting 5 folds for each of 32 candidates, totalling 160 fits
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Fitting 5 folds for each of 32 candidates, totalling 160 fits


,random_state,Prefrontal_Left_Temporal_RF_acc,Prefrontal_Left_Temporal_RF_std
0,1,0.732143,0.097153
1,2,0.814286,0.104491


137.5036060810089

In [53]:
param_grid_reduced = { 
    'n_estimators': [1, 3, 10],
    'max_features': [None, 'sqrt'],
    'max_depth' : [3,5,8, None],
    'criterion' :['gini'] }

In [54]:
t1 = time.time()

X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 2)
results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : GridSearchCV( RandomForestClassifier(), param_grid_reduced, refit = True, verbose = 1)}, return_df = True , subset_names_and_cols = Prefrontal_Left_Temporal_dict, random_states = [1,2] ) 
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
display(pd.DataFrame(res_df))
        
t2 = time.time()

t2-t1

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits


,random_state,Prefrontal_Left_Temporal_RF_acc,Prefrontal_Left_Temporal_RF_std
0,1,0.821429,0.098457
1,2,0.703571,0.242647


8.449946165084839